### Importy bibliotek

In [ ]:
import os
import json
import string
import morfeusz2
import numpy as np
import pandas as pd
import operator as op
import itertools as it
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


1. Utworzenie korpusu dokumentów

In [9]:
corpus_dir = "./literatura"
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
files = corpus.fileids()
files

['7 Krolestw- Karmazynowa Korona.txt',
 '7 Krolestw- Krol Demon.txt',
 '7 Krolestw- Tron Szarych Wilkow.txt',
 '7 Krolestw- Wygnana Krolowa.txt',
 'Diuna - Bog Imperator.txt',
 'Diuna - Dzieci.txt',
 'Diuna - Heretycy.txt',
 'Diuna - Kapitularz.txt',
 'Diuna - Mesjasz.txt',
 'Diuna.txt',
 'Kataklizm.txt',
 'Ognisty Boj.txt',
 'Saga Burzowego Archiwum - Droga Krolow.txt',
 'Saga Burzowego Archiwum - Slowa Swiatlosci.txt',
 'Saga Burzowego Archiwum -Dawca Przysiegi.txt',
 'Saga Burzowego Archiwum -Przysiega Krolowej.txt',
 'Stalowe Serce.txt',
 'Władca Pierscieni - Druzyna Pierscienia.txt',
 'Władca Pierscieni - Dwie Wieze.txt',
 'Władca Pierscieni - Powrot Krola.txt']

2. Wstępne przygotowanie dokumentów

In [10]:
documents = {}
for f in files:
    documents[f] = corpus.raw(f)
print(json.dumps(documents, indent=4, ensure_ascii=False))

{
    "7 Krolestw- Karmazynowa Korona.txt": "Po koronacji Raisa staje się królową królestwa podzielonego i pogrążonego w niepokojach. Bayarowie, mimo porażek, wciąż próbują zdominować Wizard Council. Han jako oficjalny doradca królowej zyskuje coraz więcej wpływów, ale musi stawić czoła nie tylko magii, lecz również dworskim intrygom i nieufności Clans.\r\n\r\nRaisa walczy z presją politycznych małżeństw oraz zagrożeniami zewnętrznymi – sąsiednie królestwa planują inwazję, a wewnętrzne napięcia grożą wojną domową. Tymczasem Han odkrywa prawdę o początkach konfliktu sprzed tysiąca lat, o Królowej Wilka i Demon Kingu, których losy okazują się powiązane z nim i Raisą. Legenda i historia zaczynają splatać się z teraźniejszością.\r\n\r\nW obliczu zbliżającego się konfliktu Raisa i Han muszą zjednoczyć królestwo. Ich osobiste uczucia zostają wystawione na próbę, a decyzje polityczne zagrażają ich przyszłości. Han, posługując się zarówno magią, jak i strategiami politycznymi, demaskuje zdradę

In [11]:
stoplist_file = open("./stopwords_pl.txt", "r", encoding="UTF-8")
stoplist = stoplist_file.read().splitlines()
stoplist_file.close()
stoplist = stoplist[4:]
stoplist

['vol',
 'o.o.',
 'mgr',
 'godz',
 'zł',
 'www',
 'pl',
 'ul',
 'tel',
 'hab',
 'prof',
 'inż',
 'dr',
 'i',
 'u',
 'aby',
 'albo',
 'ale',
 'ani',
 'aż',
 'bardzo',
 'bez',
 'bo',
 'bowiem',
 'by',
 'byli',
 'bym',
 'był',
 'była',
 'było',
 'były',
 'być',
 'będzie',
 'będą',
 'chce',
 'choć',
 'co',
 'coraz',
 'coś',
 'czy',
 'czyli',
 'często',
 'dla',
 'do',
 'gdy',
 'gdyby',
 'gdyż',
 'gdzie',
 'go',
 'ich',
 'im',
 'inne',
 'iż',
 'ja',
 'jak',
 'jakie',
 'jako',
 'je',
 'jednak',
 'jednym',
 'jedynie',
 'jego',
 'jej',
 'jest',
 'jeszcze',
 'jeśli',
 'jeżeli',
 'już',
 'ją',
 'kiedy',
 'kilku',
 'kto',
 'która',
 'które',
 'którego',
 'której',
 'który',
 'których',
 'którym',
 'którzy',
 'lat',
 'lecz',
 'lub',
 'ma',
 'mają',
 'mamy',
 'mi',
 'miał',
 'mimo',
 'mnie',
 'mogą',
 'może',
 'można',
 'mu',
 'musi',
 'na',
 'nad',
 'nam',
 'nas',
 'nawet',
 'nic',
 'nich',
 'nie',
 'niej',
 'nim',
 'niż',
 'no',
 'nowe',
 'np',
 'nr',
 'o',
 'od',
 'ok',
 'on',
 'one',
 'oraz',
 '

In [12]:
def lemmatize(text):
    morf = morfeusz2.Morfeusz()
    segments = it.groupby(morf.analyse(text), op.itemgetter(0,1))
    def disambiguate(group):
        pairs = ((len(descr), lemma)
                 for _, _, (_, lemma, descr, _, _, ) in group)
        perpl, lemma = min(pairs)
        return lemma.split(":")[0]
    lemmas = (disambiguate(group) for key, group in segments)
    return " ".join(filter(str.isalpha, lemmas))

In [13]:
for key in documents:
    documents[key] = documents[key].lower()
    documents[key] = "".join([char for char in documents[key] if char not in string.punctuation])
    documents[key] = lemmatize(documents[key])
    documents[key] = " ".join([word for word in word_tokenize(documents[key], language='polish') if word not in stoplist])
print(json.dumps(documents, indent=4, ensure_ascii=False))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/polish/[0m

  Searched in:
    - 'C:\\Users\\marek/nltk_data'
    - 'c:\\Users\\marek\\Desktop\\knowledge-graph-llm-new-main\\.venv\\nltk_data'
    - 'c:\\Users\\marek\\Desktop\\knowledge-graph-llm-new-main\\.venv\\share\\nltk_data'
    - 'c:\\Users\\marek\\Desktop\\knowledge-graph-llm-new-main\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\marek\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


3. Utworzenie macierzy częstości

In [ ]:
docs = pd.DataFrame.from_dict(documents, orient='index')
docs.columns = ['content']
docs

In [ ]:
count_vectorizer = CountVectorizer()
counts_tf = count_vectorizer.fit_transform(docs['content'])
counts_tf.toarray()

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
counts_tfidf = tfidf_vectorizer.fit_transform(docs['content'])
counts_tfidf.toarray()

4. Chmury tagów

In [ ]:
wordcloud = WordCloud(
    background_color='white',
    max_words=5000,
    contour_width=3,
    contour_color='steelblue'
)
if not os.path.exists("./chmury"):
    os.mkdir("./chmury")
for index, row in docs.iterrows():
    wordcloud.generate(row['content'])
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.title(index.replace(".txt", ""))
    plt.savefig("./chmury/{}".format(index.replace(".txt", ".png")))


5. Topic modelling

In [ ]:
if not os.path.exists("./tematy"):
    os.mkdir("./tematy")
def plot_top_words(model, feature_names, n_top_words, title, subplots):
    fig, axes = plt.subplots(*subplots, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.savefig(f"./tematy/{title}.png")
    plt.close()

In [ ]:
def plot_documents(model, counts, files_names, n_components, title):
  colors = ['forestgreen', 'lightskyblue', 'hotpink', 'turquoise', 'steelblue', 'crimson', 'seagreen', 'orange']
  docs_topics = pd.DataFrame(model.transform(counts), columns=[f"Topic {x}" for x in range(n_components)])
  docs_topics.index = [file_name.replace(".txt", "") for file_name in files_names]
  plt.figure(figsize=(7,4))
  left = [0] * len(docs_topics)
  for i, col in enumerate(docs_topics.columns):
    plt.barh(docs_topics.index, docs_topics[col], left=left, label=col, color = colors[i])
    left = [left[j]+docs_topics[col].iloc[j] for j in range(len(docs_topics))]
  plt.savefig(f"./tematy/{title}_docs.png")
  plt.close()

In [ ]:
n_components = 3
feature_names = count_vectorizer.get_feature_names_out()
n_top_words = 20
subplots = (1,3)

In [ ]:
lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method='online',
    learning_offset=50,
    random_state=0
)
lda.fit(counts_tf)
plot_top_words(lda, feature_names, n_top_words, "Tematy w modelu LDA", subplots)
plot_documents(lda, counts_tf, files, n_components, "Tematy w modelu LDA")

In [ ]:
nmf_fn = NMF(
    n_components=n_components,
    random_state=1,
    alpha_H=.00005,
    alpha_W=.00005,
    l1_ratio=.5
)
nmf_fn.fit(counts_tfidf)
plot_top_words(nmf_fn, feature_names, n_top_words, "Tematy w modelu NMF (FN)", subplots)
plot_documents(nmf_fn, counts_tfidf, files, n_components, "Tematy w modelu NMF (FN)")

In [ ]:
nmf_kl = NMF(
    n_components=n_components,
    random_state=1,
    beta_loss='kullback-leibler',
    solver='mu',
    max_iter=1000,
    alpha_H=.00005,
    alpha_W=.00005,
    l1_ratio=.5
)
nmf_kl.fit(counts_tfidf)
plot_top_words(nmf_kl, feature_names, n_top_words, "Tematy w modelu NMF (KL)", subplots)
plot_documents(nmf_kl, counts_tfidf, files, n_components, "Tematy w modelu NMF (KL)")

6. Analiza skupień

In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
if not os.path.exists("./skupienia"):
    os.mkdir("./skupienia")
colors = ['forestgreen', 'lightskyblue', 'hotpink', 'turquoise', 'steelblue', 'crimson', 'seagreen', 'orange']

In [ ]:
cs = cosine_similarity(counts_tfidf, counts_tfidf).flatten().reshape(20,20)
cs

In [ ]:
clustering = AgglomerativeClustering(
    n_clusters=6,
    metric="euclidean",
    linkage="ward",
    compute_distances=True
).fit(cs)
clustering.labels_

In [ ]:
plot_dendrogram(
    clustering,
    labels = files,
    truncate_mode = 'level',
    orientation = 'right',
)
plt.savefig("./skupienia/cosine_ward.png")
plt.close()

In [ ]:
ed = euclidean_distances(counts_tf, counts_tf).flatten().reshape(20,20)
ed

In [ ]:
clustering = AgglomerativeClustering(
    n_clusters=6,
    metric="precomputed",
    linkage="complete",
    compute_distances=True
).fit(ed)
clustering.labels_

In [ ]:
plot_dendrogram(
    clustering,
    labels = files,
    truncate_mode = 'level',
    orientation = 'right',
)
plt.savefig("./skupienia/euclidean_complete.png")
plt.close()

7. N-gramy

In [ ]:
documents_tokenized = {}
for key in documents:
    documents_tokenized[key] = word_tokenize(documents[key], language='polish')
print(json.dumps(documents_tokenized, indent=4, ensure_ascii=False))

In [ ]:
if not os.path.exists("./ngramy"):
    os.mkdir("./ngramy")
n_count = 3
if not os.path.exists(f"./ngramy/{n_count}"):
    os.mkdir(f"./ngramy/{n_count}")
for title in documents_tokenized:
    n_gram = pd.Series(ngrams(documents_tokenized[title], n_count)).value_counts()
    n_gram[:5].plot.barh()
    plt.savefig(f"./ngramy/{n_count}/{title.replace('.txt', '.png')}")
    plt.close()

In [ ]:
texts = " ".join(docs['content']).split(" ")
n_gram = pd.Series(ngrams(texts, 3)).value_counts()
n_gram[:15].plot.barh()
plt.gcf().subplots_adjust(left=0.4)
plt.savefig("./ngramy/all.png")
plt.close()